In [1]:
import pydicom #read dicom files
import os
import pandas as pd

In [2]:
data_dir = 'C:/Users/casti/Documents/Final_Project/FFE_images/'

In [3]:
patients = os.listdir(data_dir)

In [4]:
labels_df = pd.read_csv('C:/Users/casti/Documents/Final_Project/animallist.csv', index_col=0)

In [5]:
labels_df.head()

,tumor_model
ID,
BN13,0
BN15A,0
BN15B,0
BN15C,0
BN15D,0


In [6]:
for patient in patients:
    label = labels_df.get_value(patient, 'tumor_model')
    path = data_dir + patient
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    print(patient, len(slices), slices[0].pixel_array.shape)
#    print(slices[0])

C:\Users\casti\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


BN13 40 (192, 192)
BN15A 20 (192, 192)
BN15B 40 (192, 192)
BN15C 30 (192, 192)
BN15D 30 (192, 192)
BN17A 20 (192, 192)
BN17B 20 (192, 192)
BN19A 20 (192, 192)
BN19B 30 (192, 192)
BN20A 35 (192, 192)
BN20B 20 (192, 192)
BN21A 30 (192, 192)
BN22A 30 (192, 192)
BN23A 40 (192, 192)
BN23B 40 (192, 192)
BN24A 50 (160, 160)
BN24B 50 (160, 160)
BN24C 30 (192, 192)
BN27A 50 (192, 192)
BN27B 30 (192, 192)
BN28A 35 (192, 192)
BN28B 35 (192, 192)
BN28C 20 (192, 192)
BN29A 70 (192, 192)
BN29B 30 (224, 224)
BN30A 40 (192, 192)
BN30B 20 (192, 192)
BN30C 40 (192, 192)
BN31A 40 (192, 192)
BN31B 40 (192, 192)
BN31C 35 (192, 192)
BN31D 35 (192, 192)
BN31E 35 (160, 160)
BN31F 35 (160, 160)
BN33A 40 (192, 192)
BN33B 40 (192, 192)
BN34A 40 (192, 192)
BN34B 40 (192, 192)
BN35A 40 (192, 192)
BN35B 40 (192, 192)
BN41A 40 (192, 192)
BN41B 40 (192, 192)
BN46A 40 (192, 192)
BN46B 40 (192, 192)
BN49A 30 (192, 192)
BN50A 40 (192, 192)
BN50B 40 (192, 192)
BN51A 40 (192, 192)
BN51B 40 (192, 192)
BN52A 40 (192, 192)
B

In [7]:
len(patients)

71

In [8]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import math
%matplotlib inline

In [9]:
IMG_PX_SIZE = 50
HM_SLICES = 20

def chunks(l, n):
    #yield succesive n-sized chunks from l. source: Ned Batchelder.
    for i in range(0, len(l), n):
        yield l[i:i+n]
        
def mean(l):
    return sum(l)/len(l)

def process_data(patient, labels_df, img_px_size=50, hm_slices=20, visualize=False):
    
    label = labels_df.get_value(patient, 'tumor_model')
    path = data_dir + patient
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    
    new_slices = []
    
    slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE, IMG_PX_SIZE)) for each_slice in slices]
    
    chunk_sizes = math.ceil(len(slices) / HM_SLICES)
    
    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

    if len(new_slices) == HM_SLICES-1:
        new_slices.append(new_slices[-1])
        
    if len(new_slices) == HM_SLICES-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
    
    if len(new_slices) == HM_SLICES-3:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        
    if len(new_slices) == HM_SLICES-4:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        
    if len(new_slices) == HM_SLICES-5:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])
        
    if len(new_slices) == HM_SLICES+2:
        new_val=list(map(mean), zip(*[new_slices[HM_SLICES-1], new_slices[HM_SLICES]]))
        del new_slices[HM_SLICES]
        new_slices[HM_SLICES-1]=new_val
        
    if len(new_slices) == HM_SLICES+2:
        new_val=list(map(mean), zip(*[new_slices[HM_SLICES-1], new_slices[HM_SLICES]]))
        del new_slices[HM_SLICES]
        new_slices[HM_SLICES-1]=new_val        

    if visualize:
        fig = plt.figure(figsize=(20,20))
        for num,each_slice in enumerate(new_slices[:1]):
            y = fig.add_subplot(4, 5, num + 1)
            y.imshow(each_slice)
        plt.show()
    
    if label == 1:
        label = np.array([0,1])
    elif label == 0:
        label = np.array([1,0])
        
    return np.array(new_slices), label

much_data = []

for num, patient in enumerate(patients):
    if num%20==0:
        print(num)
    
    try:
        img_data, label = process_data(patient, labels_df, img_px_size=IMG_PX_SIZE, hm_slices=HM_SLICES)
        much_data.append([img_data, label])
    except KeyError as e:
        print('this is unlabeled data')

np.save('muchdata-{}-{}-{}.npy'.format(IMG_PX_SIZE, IMG_PX_SIZE, HM_SLICES), much_data)
    

0


C:\Users\casti\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


20
40
60


In [10]:
much_data

[[array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         ...,
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
         

In [11]:
#much_data = np.load('muchdata-192-192-20.npy', allow_pickle=True, fix_imports=True)

In [12]:
much_data

[[array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         ...,
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
         

In [13]:
len(much_data)

71

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
trainingset, validationset = train_test_split(much_data, random_state=42, test_size=0.25)

In [16]:
len(trainingset)

53

In [17]:
len(validationset)

18

In [18]:
import tensorflow as tf

In [19]:
from tensorflow.keras.models import Model

In [20]:
import numpy as np

In [21]:
from scipy import ndimage, misc
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet#, imagenet_utils
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

In [22]:
IMG_SIZE_PX=50

In [23]:
SLICE_COUNT=20

In [24]:
n_classes=2

In [25]:
x = tf.placeholder('float')
y = tf.placeholder('float')

In [26]:
keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

In [27]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

In [28]:
def maxpool3d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

In [29]:
def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               'W_fc':tf.Variable(tf.random_normal([54080,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

In [30]:
much_data = np.load('muchdata-50-50-20.npy', allow_pickle=True)
trainingset, validationset = train_test_split(much_data, random_state=42, test_size=0.25)
train_data = trainingset
validation_data = validationset

In [31]:
len(validation_data)

18

In [32]:
x = tf.placeholder('float')
y = tf.placeholder('float')

In [33]:
x

<tf.Tensor 'Placeholder_3:0' shape=<unknown> dtype=float32>

In [34]:
prediction = convolutional_neural_network(x)
cost = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(labels=prediction,logits=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [36]:
x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')



def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               'W_fc':tf.Variable(tf.random_normal([54080,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.selu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    
    conv2 = tf.nn.selu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.selu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

def train_neural_network(x):
    
    
    much_data = np.load('muchdata-50-50-20.npy', allow_pickle=True)
    trainingset, validationset = train_test_split(much_data, random_state=42, test_size=0.25)
    train_data = trainingset
    validation_data = validationset
    
    
    
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(labels=prediction,logits=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for data in train_data:
                X = data[0]
                Y = data[1]
                _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))

train_neural_network(x)

Epoch 0 completed out of 10 loss: -180772866294.0
Epoch 1 completed out of 10 loss: -1307584963840.0
Epoch 2 completed out of 10 loss: -5619998937088.0
Epoch 3 completed out of 10 loss: -17216005894144.0
Epoch 4 completed out of 10 loss: -43226997260288.0
Epoch 5 completed out of 10 loss: -91731757727744.0
Epoch 6 completed out of 10 loss: -178800198287360.0
Epoch 7 completed out of 10 loss: -312455831355392.0
Epoch 8 completed out of 10 loss: -513705836019712.0
Epoch 9 completed out of 10 loss: -797780585480192.0
Accuracy: 0.7777778


In [ ]:
Model = Sequential()

In [ ]:
Model.add(Dense(60, input_dim=60, activation='relu'))
Model.add(Dense(8, activation='relu'))
Model.add(Dense(2, activation="sigmoid"))

In [ ]:
trainingset.to_list()

In [ ]:
Model.compile(Xtrain, Ytrain)